In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from pandas.plotting import scatter_matrix

In [11]:
train = pd.read_excel('input/train.xls')

In [12]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 10 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   id                                             1887 non-null   int64 
 1   icerik                                         1887 non-null   object
 2   Gecikti veya Dağıtıma Çıkmadı                  1887 non-null   int64 
 3   Evde yok notu düşüldü veya Kapıya Getirilmedi  1887 non-null   int64 
 4   Telefonlara Cevap Verilmedi                    1887 non-null   int64 
 5   İade Süreci                                    1887 non-null   int64 
 6   Teslim Alınmadı veya Teslim Edilmedi           1887 non-null   int64 
 7   Kötü Diyalog Veya Saygısız Tutum               1887 non-null   int64 
 8   Hasarlı veya Kayıp Paket                       1887 non-null   int64 
 9   Hijyen Kurallarına Uyulmadı                    1887 non-null   

In [13]:
train['icerik'][:5]

0    Sürat Kargo Gülnur Korkmaz İşitme Engelliyim--...
1    MNG Kargo Teslimat Yapmamak İçin Ellerinden Ge...
2    PTT Kargo Covid Önlemleri Alınmıyor--> PTT Kar...
3    PTT Kargo Ürünü Adrese Getirmeme, Müşteri Hizm...
4    Aras Kargo Kargom Yola Çıkmıyor--> Şubeye gitt...
Name: icerik, dtype: object

In [38]:
X_egitim, X_test, y_egitim, y_test = train_test_split(train.loc[:,'icerik'],train.drop(['id','icerik'],axis=1), random_state=0)

In [39]:
X_egitim.shape

(1415,)

In [40]:
y_egitim.shape

(1415, 8)

In [41]:
X_test.shape

(472,)

In [42]:
y_test.shape

(472, 8)

In [54]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=1)

In [55]:
knn.fit(X_egitim,y_egitim)

ValueError: could not convert string to float: 'Aras Kargo Ürünüm Alıcıya Ulaştırılmadı, Geri Dönen Kargomdan Ücret Talep Edildi--> 8 Nisan tarihinde 20 TL vererek aynı şehir içinde eve teslim edilmesi üzerine gönderdiğim kargom "alıcı bulunamadı" denilerek 2 ay sonra şubeye geri döndü. Covid-19 karantina günlerinde mümkün olmayan bir gerekçe ile kişi adreste bulunulmadığı (!) iddia edildi. Birinci fiyasko! Ayrıca kişinin adreste bulunamadığı durumlarda, kişi telefon ile aranır. Böyle bir arama da gerçekleşmeden, kargoyu geri göndermişler.İkinci olarak, şubeden aranıp kargomu gelip teslim almam gerektiğini, evime ulaştıramayacaklarını söylediler.Üçüncü olarak, şubeye giderek 2 aydır alıcıya teslim edilemeyen kargomu (kendi malımı) geri almak istediğimde 11 TL ücret çıkarıldı. Neden olarak da "iade" dendi. İade olmayan, kargo firmasının beceriksizliği ve savsaklığı ile ulaşmayan kargom geri geldi diye müşteriden ücret talep edilmesi üzerine müşteri hizmetlerini defalarca aradım. Şikayet